In [15]:
# https://www.youtube.com/watch?v=Z1e2fJFTdGI


# TODO
# -----
# Tempted to create an outer brown border, white inside, then image inside



from PIL import Image, ImageDraw, ImageFont, ImageOps
import os

border_as_percentage_of_image_longest_side = 0.01

in_path = r'D:\Onedrive\Documents_Charl\Computer_Technical\Programming_GitHub\AddImageCaption\Landscape_Example.JPG'
out_path = r'D:\Onedrive\Documents_Charl\Computer_Technical\Programming_GitHub\AddImageCaption\out.jpg'

img = Image.open(in_path)

# If orientation is stored via exif metadata then transpose image and remove the metadata
img = ImageOps.exif_transpose(img)

width, height = img.size
max_side_lenth = max(width, height)
border_width = int(max_side_lenth * border_as_percentage_of_image_longest_side)

bi = Image.new(
    'RGB', 
    (width + border_width * 2, height + (int(height/5))), 
    'RGB(200,200,200)')

bi.paste(img, (border_width, border_width))
# bi.paste(img, (5,5, (width+5), (height+5)))

caption = 'Leaving Sydney'
font = ImageFont.truetype('arial.ttf', 45)
w,h  = font.getsize(caption)

draw = ImageDraw.Draw(bi)
draw.text(((width-w)/2, (height + ((height/5)-h)/2)), caption, font=font, fill = 'black')
bi.save(out_path)



In [14]:
height

5184